# Vertical Chat
A sample how to build a chat for small business using:

* GPT 35
* Panel
* OpenAI


This is just a simple sample to start to understand how the OpenAI API works, and how to create Prompts. It Is really far from beign a complete solution.
We are going to introduce some interesting points:

* The roles in a conversation.
* How is the conversations’ memory preserved?

Deeper explanations in the article: [Create Your First Chatbot Using GPT 3.5, OpenAI, Python and Panel.](https://medium.com/towards-artificial-intelligence/create-your-first-chatbot-using-gpt-3-5-openai-python-and-panel-7ec180b9d7f2)

In [1]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import panel as pn
import os
import asyncio

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

aclient = AsyncOpenAI(api_key=OPENAI_API_KEY)

pn.extension()

# Define the initial context with the system prompt for the OrderBot role
context = [{'role': 'system', 'content': """
Act as an OrderBot, you work collecting orders in a delivery-only fast food restaurant called
My Dear Frankfurt. First welcome the customer, in a very friendly way, then collect the order.
You wait to collect the entire order, beverages included, then summarize it and check for a final
time if everything is ok or the customer wants to add anything else. Finally, you collect the payment.
Make sure to clarify all options, extras, and sizes to uniquely identify the item from the menu.
You respond in a short, very friendly style. The menu includes:

burger  12.95, 10.00, 7.00
frankfurt   10.95, 9.25, 6.50
sandwich   11.95, 9.75, 6.75
fries 4.50, 3.50
salad 7.25
Toppings:
extra cheese 2.00
mushrooms 1.50
martra sausage 3.00
canadian bacon 3.50
romesco sauce 1.50
peppers 1.00
Drinks:
coke 3.00, 2.00, 1.00
sprite 3.00, 2.00, 1.00
vichy catalan 5.00
"""}]


async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context
    response = await aclient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        stream=True,
    )

    message = ""
    async for chunk in response:
        part = chunk.choices[0].delta.content
        if part is not None:
            message += part
            yield message


def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )

    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    pn.serve(chat_interface, show=True)


main()


Launching server at http://localhost:56168


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### Version 1
Let's try to make the bot more straightforward by clearly identifying the steps it needs to go through and tuning the parameters to avoid repetitions and too much creativity

In [5]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import panel as pn
import os
import asyncio

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

aclient = AsyncOpenAI(api_key=OPENAI_API_KEY)

pn.extension()

# Define the initial context with the system prompt for the OrderBot role
context = [{'role': 'system', 'content': """
You are OrderBot, an assistant for a delivery-only fast food restaurant called My Dear Frankfurt. Your role is to:
1. Greet the customer warmly and inquire about their order.
2. Collect the order details, including food items, extras, sizes, and drinks.
3. After collecting the complete order, confirm the items and ask if they want to add anything else.
4. Once confirmed, proceed to request payment and complete the order.

### Menu ###
Main Items:
- Burger: 12.95 (large), 10.00 (medium), 7.00 (small)
- Frankfurt: 10.95 (large), 9.25 (medium), 6.50 (small)
- Sandwich: 11.95 (large), 9.75 (medium), 6.75 (small)
- Fries: 4.50 (large), 3.50 (small)
- Salad: 7.25

Toppings:
- Extra Cheese: 2.00
- Mushrooms: 1.50
- Martra Sausage: 3.00
- Canadian Bacon: 3.50
- Romesco Sauce: 1.50
- Peppers: 1.00

Drinks:
- Coke: 3.00 (large), 2.00 (medium), 1.00 (small)
- Sprite: 3.00 (large), 2.00 (medium), 1.00 (small)
- Vichy Catalan: 5.00

Follow these steps accurately. Do not repeat yourself or re-ask questions unnecessarily. Move to payment as soon as the order is confirmed.
"""}]

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context with optimized parameters
    response = await aclient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0.5,  # Lower temperature for consistency
        frequency_penalty=0.5,  # Discourage repetitive phrasing
        presence_penalty=0,     # Encourage relevant responses
        stream=True,
    )

    message = ""
    async for chunk in response:
        part = chunk.choices[0].delta.content
        if part is not None:
            message += part
            yield message

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )

    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    pn.serve(chat_interface, show=True)

main()


Launching server at http://localhost:57343


### Version 2
In this version we focus on hallucination during the payment step by being more specific regarding the payment step and how the total price should be calculated. Once calculated, the model should refer to it and not calculate/estimate it again.

In [ ]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import panel as pn
import os
import asyncio

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

aclient = AsyncOpenAI(api_key=OPENAI_API_KEY)

pn.extension()

# Define the initial context with the system prompt for the OrderBot role
context = [{'role': 'system', 'content': """
You are OrderBot, an assistant for a delivery-only fast food restaurant called My Dear Frankfurt. Your role is to:
1. Greet the customer warmly and inquire about their order.
2. Collect the order details, including food items, extras, sizes, and drinks.
3. After collecting the complete order, confirm the items and ask if they want to add anything else.
4. Proceed to request payment and complete the order.

### Menu ###
Main Items:
- Burger: 12.95 (large), 10.00 (medium), 7.00 (small)
- Frankfurt: 10.95 (large), 9.25 (medium), 6.50 (small)
- Sandwich: 11.95 (large), 9.75 (medium), 6.75 (small)
- Fries: 4.50 (large), 3.50 (small)
- Salad: 7.25

Toppings:
- Extra Cheese: 2.00
- Mushrooms: 1.50
- Martra Sausage: 3.00
- Canadian Bacon: 3.50
- Romesco Sauce: 1.50
- Peppers: 1.00

Drinks:
- Coke: 3.00 (large), 2.00 (medium), 1.00 (small)
- Sprite: 3.00 (large), 2.00 (medium), 1.00 (small)
- Vichy Catalan: 5.00

Follow these steps accurately. Do not repeat yourself or re-ask questions unnecessarily. Move to payment as soon as the order is confirmed.
"""}]

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context with optimized parameters
    response = await aclient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0,  # Lower temperature for consistency
        frequency_penalty=0.5,  # Discourage repetitive phrasing
        presence_penalty=0,     # Encourage relevant responses
        stream=True,
    )

    message = ""
    async for chunk in response:
        part = chunk.choices[0].delta.content
        if part is not None:
            message += part
            yield message

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )

    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    pn.serve(chat_interface, show=True)

main()


INFO:bokeh.server.views.ws:WebSocket connection closed: code=1001, reason=None


### Version 3
In this version, we try to lock the payment once it has been calculated and if there are no more additions.

In [20]:
from openai import AsyncOpenAI
from dotenv import load_dotenv
import panel as pn
import os
import asyncio

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

aclient = AsyncOpenAI(api_key=OPENAI_API_KEY)

pn.extension()

# Define the initial context with the system prompt for the OrderBot role
context = [{'role': 'system', 'content': """
You are OrderBot, an assistant for a delivery-only fast food restaurant called My Dear Frankfurt. Your role is to:
1. Greet the customer warmly and inquire about their order.
2. Collect the order details, including food items, extras, sizes, and drinks.
3. After collecting the complete order, confirm the items and ask if they want to add anything else. If not, move on to step 4, payment.
4. Proceed to request payment and complete the order.

### Menu ###
Main Items:
- Burger: 12.95 (large), 10.00 (medium), 7.00 (small)
- Frankfurt: 10.95 (large), 9.25 (medium), 6.50 (small)
- Sandwich: 11.95 (large), 9.75 (medium), 6.75 (small)
- Fries: 4.50 (large), 3.50 (small)
- Salad: 7.25

Toppings:
- Extra Cheese: 2.00
- Mushrooms: 1.50
- Martra Sausage: 3.00
- Canadian Bacon: 3.50
- Romesco Sauce: 1.50
- Peppers: 1.00

Drinks:
- Coke: 3.00 (large), 2.00 (medium), 1.00 (small)
- Sprite: 3.00 (large), 2.00 (medium), 1.00 (small)
- Vichy Catalan: 5.00

Follow these steps accurately.
In all of your answers, list the ordered items and final price based on the chat history. 
Do not recalculate the final price if there is no update in the order.
Move to payment as soon as the order is confirmed.
Do not repeat yourself, once you get all information needed, move on to the next step.
"""}]

async def callback(contents: str, user: str, instance: pn.chat.ChatInterface):
    # Add the user's message to the context
    context.append({'role': 'user', 'content': contents})

    # Generate a response using the existing context with optimized parameters
    response = await aclient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=context,
        temperature=0,  # Temperature at 0
        frequency_penalty=0.5,  # Discourage repetitive phrasing
        presence_penalty=0,     # Encourage relevant responses
        stream=True,
    )

    message = ""
    async for chunk in response:
        part = chunk.choices[0].delta.content
        if part is not None:
            message += part
            yield message

def main():
    chat_interface = pn.chat.ChatInterface(
        callback=callback,
        callback_user="OrderBot",
        help_text="Welcome to My Dear Frankfurt! How can I help you with your order today?",
    )

    template = pn.template.FastListTemplate(
        title="OpenAI OrderBot",
        header_background="#212121",
        main=[chat_interface]
    )

    template.servable()
    pn.serve(chat_interface, show=True)

main()


INFO:bokeh.server.server:Starting Bokeh server version 3.4.3 (running on Tornado 6.4)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:60301


INFO:tornado.access:200 GET / (::1) 193.00ms
INFO:tornado.access:200 GET /static/js/bokeh.min.js?v=0a5da0be1c2385c8fbc778b6fe92af35da98e7b160ed40850e1b234b02df34a055ca50d37f82f056d346f1a6812c836eecbfb99371cfc264da60861011003592 (::1) 3.00ms
INFO:tornado.access:200 GET /static/js/bokeh-gl.min.js?v=2bba96f5a73c631f21cb84fc6eb04f860e9b43e809ffaf3e7c7a60109b6c6c42a9b83a3d4bdadb0cb023bd48069e8d5cb2ee624da5fa4d5839469ac6818a41af (::1) 2.00ms
INFO:tornado.access:200 GET /static/js/bokeh-widgets.min.js?v=bf4eea8535eba9536cc13c5656d74f40bbf1448e73e48ddf7eaddff850f7d0093bc141ef1da5e7579585c338db8db7bb4b306cbcde009d4dbe9af2e7c2d609fb (::1) 2.00ms
INFO:tornado.access:200 GET /static/js/bokeh-tables.min.js?v=30dd48a6c21439042b22c0586013c023610fdaba3c28ecd2549991ee689caaf69984c17f610f0259a9e7b9a0cec40dd018f52d36b47a6bd65e10d9885d3a861e (::1) 2.00ms
INFO:tornado.access:200 GET /static/extensions/panel/panel.min.js?v=d313463cd570e018205834e09df54b82ee74f424e1fb8b438a7ebf8855ebba25 (::1) 2.00ms
INFO:to

## Conclusions

**Version 0:**  
This version is quite buggy. The chatbot is repeating over and over the helpline. It is able to take my order but some items suddenly appear or disappear. I am not able to reach the payment step, as it keeps asking me if I need something else.

**Version 1:**
This version is better. The chatbot messages are more structured using bullet points, clear and more direct answers. Still some hallucinations when it comes to payment, it does not remember the exact price between two consecutive messages.

**Version 2:**
This version seemed better at first, going over the orders and calculating the final price in a very clear way. But then it refers to one of the item only and eventually asks the client to pay for this single item in the end, forgetting the total price.

**Version 3:**  
In this final version, we are close to a reliable chatbot. It is stating the content of the basket and final price at every step. It is moving to the payment step once we reach it and is able to process with the payment without altering the list of items and the final price. However, if forced to go back (example: I forgot, I also want a small coke), it gets confused, adds the item but does not update the final price.

In the end, the best approach would probably be using a json list or dict to store the basket content and prices. Final price should be calculated automatically everytime we update the basket, so the LLM is only reading and updating its contents.